In [6]:
import pandas as pd
import re
from datetime import datetime

current_year = datetime.now().year
print(current_year)

2024


In [2]:
class Utils:

  @staticmethod
  def to_lower_case_non_accent_vietnamese(s: str) -> str:
    s = s.lower()
    s = re.sub(r'à|á|ạ|ả|ã|â|ầ|ấ|ậ|ẩ|ẫ|ă|ằ|ắ|ặ|ẳ|ẵ', 'a', s)
    s = re.sub(r'è|é|ẹ|ẻ|ẽ|ê|ề|ế|ệ|ể|ễ', 'e', s)
    s = re.sub(r'ì|í|ị|ỉ|ĩ', 'i', s)
    s = re.sub(r'ò|ó|ọ|ỏ|õ|ô|ồ|ố|ộ|ổ|ỗ|ơ|ờ|ớ|ợ|ở|ỡ', 'o', s)
    s = re.sub(r'ù|ú|ụ|ủ|ũ|ư|ừ|ứ|ự|ử|ữ', 'u', s)
    s = re.sub(r'ỳ|ý|ỵ|ỷ|ỹ', 'y', s)
    s = re.sub(r'đ', 'd', s)
    s = re.sub(r'\u0300|\u0301|\u0303|\u0309|\u0323', '', s)  # Huyền sắc hỏi ngã nặng
    s = re.sub(r'\u02C6|\u0306|\u031B', '', s)  # Â, Ê, Ă, Ơ, Ư
    return s

  @staticmethod
  def to_non_accent_vietnamese(s: str, lower = True) -> str:
    s = re.sub(r'A|Á|À|Ã|Ạ|Â|Ấ|Ầ|Ẫ|Ậ|Ă|Ắ|Ằ|Ẵ|Ặ', 'A', s)
    s = re.sub(r'à|á|ạ|ả|ã|â|ầ|ấ|ậ|ẩ|ẫ|ă|ằ|ắ|ặ|ẳ|ẵ', 'a', s)
    s = re.sub(r'E|É|È|Ẽ|Ẹ|Ê|Ế|Ề|Ễ|Ệ', 'E', s)
    s = re.sub(r'è|é|ẹ|ẻ|ẽ|ê|ề|ế|ệ|ể|ễ', 'e', s)
    s = re.sub(r'I|Í|Ì|Ĩ|Ị', 'I', s)
    s = re.sub(r'ì|í|ị|ỉ|ĩ', 'i', s)
    s = re.sub(r'O|Ó|Ò|Õ|Ọ|Ô|Ố|Ồ|Ỗ|Ộ|Ơ|Ớ|Ờ|Ỡ|Ợ', 'O', s)
    s = re.sub(r'ò|ó|ọ|ỏ|õ|ô|ồ|ố|ộ|ổ|ỗ|ơ|ờ|ớ|ợ|ở|ỡ', 'o', s)
    s = re.sub(r'U|Ú|Ù|Ũ|Ụ|Ư|Ứ|Ừ|Ữ|Ự', 'U', s)
    s = re.sub(r'ù|ú|ụ|ủ|ũ|ư|ừ|ứ|ự|ử|ữ', 'u', s)
    s = re.sub(r'Y|Ý|Ỳ|Ỹ|Ỵ', 'Y', s)
    s = re.sub(r'ỳ|ý|ỵ|ỷ|ỹ', 'y', s)
    s = re.sub(r'Đ', 'D', s)
    s = re.sub(r'đ', 'd', s)
    s = re.sub(r'\u0300|\u0301|\u0303|\u0309|\u0323', '', s)  # Huyền sắc hỏi ngã nặng
    s = re.sub(r'\u02C6|\u0306|\u031B', '', s)  # Â, Ê, Ă, Ơ, Ư
    if lower:
      s = re.sub('[^a-z0-9]','',s.replace(' ',"").lower())
    return s

In [8]:
df = pd.read_excel('member.xlsx', na_filter=False)
df = df.drop(columns=['Dấu thời gian'])
remame_mapping = {
    'Họ và tên': 'name',
    'personal email': 'personal email',
    'năm tốt nghiệp (hoặc dự kiến tốt nghiệp)': 'graduation year',
    "Alias\n* Các cách ghi tên của bản thân trên các publication hoặc các research\n* Điền theo cú pháp mỗi alias trên 1 dòng\nvd:\nNguyen Van A\nA Nguyen\nA Van Nguyen\nVan-A Nguyen": 'alias',
    'affiliation': 'affiliation',
    'Các thông tin liên lạc (email, linkedin, profile page,...)\n* Viết theo cú pháp mẫu sau:\nemail: ml4u@example.email\nlinkedin: http://linkedin.com\ngoogle-scholar: exampleid123\ngithub: my_git_hut_username\nhomepage: myhomepage.com\n........': 'contact',
    'Ảnh\n*Kiếm ảnh nào ngầu ngầu vào\n* Đặt tên file ảnh là: Họ và tên\nvd: NguyenVanA.jpg': 'image',
    'short description bản thân': 'short description',
    'long description bản thân\n*must in tiếng anh\n*nếu viết dưới dạng markdown thì thật tuyệt vời\nvd:\n```\nI currently study in KAIST EE as a Master student.\nFrom March 2023, I work at Multimodal AI Lab under the supervision of [Prof. Joon Son Chung](https://mmai.io/joon)\n```': 'long description',
    'personal page (nếu có)': 'personal page',
    'ORCID': 'orcid',
    'Publication:\n* viết dưới dạng doi và ảnh\n*vd:\n\nid: doi:10.1007/978-3-030-98015-3_10\nimage: https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-3-030-98015-3_10/MediaObjects/518004_1_En_10_Fig2_HTML.png?as=webp\n id: doi:10.1007/978-3-030-97610-1_6\nimage: https://signofthefour.github.io/images/CalibStyelSpeech.png': 'publication',
    'Hướng nghiên cứu': 'research direction'
}
df = df.rename(columns=remame_mapping)
df

,name,personal email,graduation year,alias,affiliation,contact,image,short description,long description,personal page,orcid,publication,research direction
0,Trần Trung Thái,thaitran5307@gmail.com,2024,Trung Thai Tran,Ho Chi Minh City University of Technology (HCM...,email: thaitran5307@gmail.com\nemail: thai.tra...,https://drive.google.com/open?id=1J0bKmCft87JZ...,,Enthusiastic and dedicated researcher in the f...,,0009-0002-1422-9685,id: doi: 10.1007/978-3-031-46573-4_39\nid: doi...,Computer Vision
1,Trần Nguyễn Phương Thành,tnphthanh@gmail.com,2025,Phuong Thanh Tran Nguyen,,email: tnphthanh@gmail.com\nlinkedin: https://...,https://drive.google.com/open?id=17MbtBtreVHaC...,Vietnamese Computer Science student who finds ...,"```\nI currently a final year student at BKU, ...",,https://orcid.org/0009-0004-9381-4667,,Language and Speech Processing
2,Vũ Châu Duy Quang,vuchauduyquang@gmail.com,2025,Quang C. D. Vu\nQuang Vu,Ho Chi Minh City University of Technology (HCMUT),email: vuchauduyquang@gmail.com\nlinkedin: htt...,https://drive.google.com/open?id=1PpXVtkW5XF4B...,,I'm currently an undergraduate in HCMUT.,,https://orcid.org/0000-0003-2189-6259,,Computer Vision
3,Lê Minh Nghĩa,minhnghiale1590@gmail.com,2024,Minh Nghia Le,,email: minhnghiale1590@gmail.com\ngithub: LMN1590,https://drive.google.com/open?id=1WcK-xpuFocqf...,,I received my Bachelor's degree in Computer Sc...,,,,Computer Vision
4,Thẩm Quốc Dũng,thamquocdung@gmail.com,2019,Quoc Dung Tham\nQuoc-Dung Tham,Computer Science and Engineering Faculty - Ho ...,email: thamquocdung@gmail.com\nlinkedin: https...,https://drive.google.com/open?id=13tnNVx0bMU7X...,I am a senior AI Engineer with over five years...,"Currently, I am a Senior Applied AI Scientist ...",,,id: doi:10.1109/RIVF60135.2023.10471783\nimage...,Language and Speech Processing
5,Nguyen Quan Anh Minh,nguyenquananhminh@gmail.com,2019,Quan Anh Minh Nguyen,,https://www.linkedin.com/in/dathudeptrai/,https://drive.google.com/open?id=1vsB_NwpWpvxE...,"tall, big, black",I currently a super AI engineer at speechify,,,https://dl.acm.org/doi/10.1007/978-3-030-33709...,"Language and Speech Processing, Natural Langua..."


In [25]:
for row in df.iterrows():
    newline = '\n'
    row = row[1]
    name = row['name']
    filename = Utils.to_lower_case_non_accent_vietnamese(name).replace(' ','')+'.md'
    alias = row['alias']
    year_of_graduation = row['graduation year']
    affiliation = row['affiliation']
    role = 'alumni' if year_of_graduation <= current_year else 'undergrad'
    short_description = row['short description'] if row['short description'] else 'graduate in '+str(year_of_graduation) if role == 'alumni' else 'team member'
    long_description = row['long description']
    contact = row['contact']
    orcid = row['orcid']
    context = f"""---
name: {name}
image: 
description: {short_description}
role: {role}
aliases:
{f'{newline}'.join(list(map(lambda x: ' - '+x,alias.split(newline))))}
affiliation: {affiliation}
links:
{f'{newline}'.join(list(map(lambda x: ' '+x ,contact.split(newline))))}
 {f'orcid: {orcid}' if orcid else ''}
 {f'home-page: {row["personal page"]}' if row["personal page"] else ''}
---

{long_description}
"""
    context = re.sub(r"(https:\/\/www\.linkedin\.com\/in\/)?([a-zA-Z0-9-]+)\/?",r"\2", context)
    context = re.sub(r"(https:\/\/github\.com\/)?([a-zA-Z0-9-]+)\/?",r"\2", context)
    context = re.sub(r"(https:\/\/orcid\.org\/)?([0-9]{4}-[0-9]{4}-[0-9]{4}-[0-9]{4})\/?",r"\2", context)
    with open(f"_members/{filename}", "w", encoding='utf-8') as f:
        f.write(context)